In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
con = create_engine("sqlite:///summary.sqlite")

In [3]:
con.table_names()

['api',
 'pivot',
 'ratio0',
 'ratio1',
 'ratioapi',
 'regression',
 'rent0',
 'rent1',
 'sales0',
 'sales1']

In [18]:
sales1 = pd.read_sql("sales1",con)

In [19]:
sales1

,index,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
0,intercept,2.162541,0.673941,3.208799,0.014883,0.568924,3.756159
1,population,-0.005129,0.002277,-2.252710,0.058962,-0.010512,0.000255
2,density,0.004444,0.005514,0.806021,0.446744,-0.008594,0.017483
3,violent crime,-7.371075,7.778942,-0.947568,0.374905,-25.765349,11.023199
4,property crime,4.895421,7.016935,0.697658,0.507894,-11.696993,21.487834
5,MCDONALDS_count,12.968615,77.069829,0.168271,0.871129,-169.272572,195.209803
6,RESTAURANT_count,43.250826,13.478821,3.208799,0.014883,11.378480,75.123173
7,STARBUCKS_count,28.336558,17.542833,1.615278,0.150285,-13.145649,69.818766
8,SUPER DUPER BURGER_count,-139.220861,216.357675,-0.643475,0.540415,-650.825467,372.383744
9,MCDONALDS_mean,-0.067468,0.081184,-0.831047,0.433375,-0.259438,0.124502


In [29]:
def hl_regr(x):
    c0 = 'border-color: black'
    c1 = 'background-color: lightgreen'
    c2 = 'background-color: lightblue'
    c3 = 'background-color: yellow'
    c4 = 'background-color: pink'
    c5 = 'background-color: orange'
    c6 = 'background-color: red'
#     g1 = 'background-color: lightblue'
#     g2 = 'background-color: pink'
    #if want set no default colors 
    #c2 = ''  
    m1 = x["P>|t|"] > 0
    m2 = x["P>|t|"] > .05
    m3 = x["P>|t|"] > .1
    m4 = x["P>|t|"] > .2
    m5 = x["P>|t|"] > .3
    m6 = x["P>|t|"] > .5
#     mg1 = x["gender"] == 1
#     mg2 = x["gender"] == 2

    df1 = pd.DataFrame(c0,index=x.index, columns=x.columns)

    df1.loc[m1, 'P>|t|'] = c1
    df1.loc[m2, 'P>|t|'] = c2
    df1.loc[m3, 'P>|t|'] = c3
    df1.loc[m4, 'P>|t|'] = c4
    df1.loc[m5, 'P>|t|'] = c5
    df1.loc[m6, 'P>|t|'] = c6
#     df1.loc[mg1, 'name'] = g1
#     df1.loc[mg2, 'name'] = g2
    return df1

In [30]:
# sales1.index+=1
sales0 = sales1.style.apply(hl_regr, axis=None)

In [31]:
sales0

,index,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
0,intercept,2.16254,0.673941,3.2088,0.0148829,0.568924,3.75616
1,population,-0.00512857,0.00227662,-2.25271,0.0589619,-0.0105119,0.000254786
2,density,0.00444439,0.00551399,0.806021,0.446744,-0.00859412,0.0174829
3,violent crime,-7.37107,7.77894,-0.947568,0.374905,-25.7653,11.0232
4,property crime,4.89542,7.01693,0.697658,0.507894,-11.697,21.4878
5,MCDONALDS_count,12.9686,77.0698,0.168271,0.871129,-169.273,195.21
6,RESTAURANT_count,43.2508,13.4788,3.2088,0.0148829,11.3785,75.1232
7,STARBUCKS_count,28.3366,17.5428,1.61528,0.150285,-13.1456,69.8188
8,SUPER DUPER BURGER_count,-139.221,216.358,-0.643475,0.540415,-650.825,372.384
9,MCDONALDS_mean,-0.067468,0.0811843,-0.831047,0.433375,-0.259438,0.124502
